In [6]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, balanced_accuracy_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from diffusers import UNet2DModel, DDIMScheduler, DDPMScheduler, LMSDiscreteScheduler
import torch
import torch.nn as nn
import torch.optim as optim
import csv

In [8]:
df_IoT=pd.read_csv("IoTDIAD.csv")
df_IoT

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,192.168.137.66-192.168.137.174-41082-80-6,192.168.137.66,41082.0,192.168.137.174,80.0,6.0,09/08/2022 11:27:38 AM,1527173.0,16.0,1.0,...,20.0,0.000000e+00,0.000000e+00,0.0,0.0,0.00,0.000000,0.0,0.0,DoS
1,192.168.137.66-192.168.137.254-55598-34287-6,192.168.137.66,55598.0,192.168.137.254,34287.0,6.0,09/08/2022 11:56:16 AM,9912071.0,7.0,2.0,...,32.0,0.000000e+00,0.000000e+00,0.0,0.0,0.00,0.000000,0.0,0.0,DoS
2,192.168.137.174-192.168.137.66-80-47994-6,192.168.137.174,80.0,192.168.137.66,47994.0,6.0,09/08/2022 11:29:10 AM,0.0,2.0,0.0,...,20.0,0.000000e+00,0.000000e+00,0.0,0.0,0.00,0.000000,0.0,0.0,DoS
3,192.168.137.66-192.168.137.254-59336-8009-6,192.168.137.66,59336.0,192.168.137.254,8009.0,6.0,09/08/2022 11:50:16 AM,349868.0,1.0,1.0,...,40.0,0.000000e+00,0.000000e+00,0.0,0.0,0.00,0.000000,0.0,0.0,DoS
4,192.168.137.66-192.168.137.254-55662-8009-6,192.168.137.66,55662.0,192.168.137.254,8009.0,6.0,09/08/2022 11:49:16 AM,215841.0,1.0,1.0,...,40.0,0.000000e+00,0.000000e+00,0.0,0.0,0.00,0.000000,0.0,0.0,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33613,192.168.137.79-255.255.255.255-49154-6667-17,192.168.137.79,49154.0,255.255.255.255,6667.0,17.0,07/10/2022 03:06:15 PM,119995219.0,25.0,0.0,...,8.0,1.165906e+07,6.061784e+06,19997148.0,4965723.0,5006391.25,13272.949739,5038906.0,5000281.0,BenignTraffic
33614,157.249.81.141-192.168.137.41-80-51746-6,157.249.81.141,80.0,192.168.137.41,51746.0,6.0,07/10/2022 06:00:51 PM,126325.0,4.0,0.0,...,32.0,0.000000e+00,0.000000e+00,0.0,0.0,0.00,0.000000,0.0,0.0,BenignTraffic
33615,192.168.137.186-192.168.137.1-42090-53-17,192.168.137.186,42090.0,192.168.137.1,53.0,17.0,07/10/2022 05:16:07 PM,85450.0,2.0,2.0,...,8.0,0.000000e+00,0.000000e+00,0.0,0.0,0.00,0.000000,0.0,0.0,BenignTraffic
33616,205.174.165.69-192.168.137.187-24421-63908-17,205.174.165.69,24421.0,192.168.137.187,63908.0,17.0,07/10/2022 09:11:08 PM,1115118.0,7.0,0.0,...,8.0,0.000000e+00,0.000000e+00,0.0,0.0,0.00,0.000000,0.0,0.0,BenignTraffic


In [9]:
df_IoT['label'].unique()

array(['DoS', 'Mirai', 'Recon', 'Spoofing', 'Web-based', 'Brute_Force',
       'DDoS', 'BenignTraffic'], dtype=object)

In [ ]:
# Đọc file thứ hai chứa cột tên và số
# Giả sử file thứ hai tên là 'class_names.csv' với cột 'ten' và 'so'
df_map = pd.read_csv('class_names.csv')

# Tạo dictionary ánh xạ từ tên sang số
name_to_number = dict(zip(df_map['Class Name'], df_map['Class Label']))

# Thay thế cột tên trong df_IoT bằng số tương ứng
# Giả sử cột tên trong df_IoT cũng tên là 'ten'
print(name_to_number)
 n  
df_IoT.head()

{'BenignTraffic': 0, 'Brute_Force': 1, 'DDoS': 2, 'DoS': 3, 'Mirai': 4, 'Recon': 5, 'Spoofing': 6, 'Web-based': 7}


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,192.168.137.66-192.168.137.174-41082-80-6,192.168.137.66,41082.0,192.168.137.174,80.0,6.0,09/08/2022 11:27:38 AM,1527173.0,16.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS
1,192.168.137.66-192.168.137.254-55598-34287-6,192.168.137.66,55598.0,192.168.137.254,34287.0,6.0,09/08/2022 11:56:16 AM,9912071.0,7.0,2.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS
2,192.168.137.174-192.168.137.66-80-47994-6,192.168.137.174,80.0,192.168.137.66,47994.0,6.0,09/08/2022 11:29:10 AM,0.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS
3,192.168.137.66-192.168.137.254-59336-8009-6,192.168.137.66,59336.0,192.168.137.254,8009.0,6.0,09/08/2022 11:50:16 AM,349868.0,1.0,1.0,...,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS
4,192.168.137.66-192.168.137.254-55662-8009-6,192.168.137.66,55662.0,192.168.137.254,8009.0,6.0,09/08/2022 11:49:16 AM,215841.0,1.0,1.0,...,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoS
